In [ ]:
from bs4 import BeautifulSoup as bs
from urllib import request
from urllib.request import quote
import urllib
import requests
import pandas as pd
import re
import os
import sys
import json

In [ ]:
# 네이버 API 로그인 후 link 정보 가져와 돌려주기
def get_Nav_client(encText, sPage):
    # 기록할 빈 파일 생성
    fq = open('c:/pydata/' + encText + '_질문.txt', "w")
    fa = open('c:/pydata/' + encText + '_답변.txt', "w")

    url = "https://openapi.naver.com/v1/search/kin?query=" + quote(encText) # json 결과
    url = url + "&start=" + str(sPage) + "&display=100"

    headers = {"X-Naver-Client-Id":"TWoFiP4PRGZBGB6H_JSm", 
               "X-Naver-Client-Secret":"TOXDluSLJh"}

    res_content=requests.get(url, headers=headers)

    if res_content.status_code == 200:
        res_json = res_content.json()
    else:
        print("Error Code:" + str(res_json.status_code))
        sys.exit(0)
    
    fq.close()
    fa.close()
    
    df=pd.DataFrame(res_json['items'])
       
    return list(df['link'])

In [ ]:
#link를 이용해 세부 내용 가져와 파일에 기록하기
def get_write(encText, link):
    kin_html = requests.get(link)

    fq = open('c:/pydata/' + encText + '_질문.txt', "a")
    fa = open('c:/pydata/' + encText + '_답변.txt', "a")

    if kin_html.status_code != 200 :
        fq.close()
        fa.close()
    else:
        kin_soup = bs(kin_html.text, "html.parser")

        # 제목 가져오기
        try:
            kin_tit = kin_soup.find("div", class_="title").get_text().replace("\n", "").replace("\t", "")
            fq.write(kin_tit+"\n")
        except:
            fq.write("\n")

        kin_body=kin_soup.find('div', class_="c-heading__content")

        # 질문 내용
        try:
            kin_body=kin_soup.find('div', class_="c-heading__content").text.replace("\n", "").replace("\t", "")
            fq.write(kin_body)
        except:
            fq.write("\n")

        # 답변 내용

        for kin_ans in kin_soup.find_all('div', class_="se-component se-text se-l-default"):
            fa.write(re.sub("[^가-힣 ]","",kin_ans.text))
            fq.write("\n")

        fq.close()
        fa.close()

In [ ]:
if __name__ == "__main__":
    #encText = urllib.parse.quote("제주도")
    encText = "제주도"

    for i in range(1, 1001,100):
    
        reg_link=get_Nav_client(encText, i)

        # link를 이용해 세부 내용 가져와 파일에 기록하기
        for link in reg_link:
            get_write(encText, link)

    print("=== End ===")

=== End ===
